In [1]:
import pandas as pd
import json
import minsearch

# Ingestion

In [2]:
with open('../data/arsonor_chunks_id.json', 'r', encoding='utf-8') as file:
    documents = json.load(file)

In [3]:
documents[15]

{'article_id': '3632a3b4',
 'title': 'L’intelligence artificielle (IA) dans le studio de production audio (5/6)',
 'category': 'LA POST-PROD',
 'tags': 'dé-mixage, de-noise, de-reverb, deep learning, plug-in audio, restauration audio, stems',
 'chunk_id': '3632a3b4-11',
 'chunk_text': 'A sa sortie en 2016, Regroover reçut plusieurs prix d’innovation et fut considéré comme le futur du sampling. Enfin, le bundle ERA , sorti en 2018, est la gamme de produits d’Accusonus qui connaît la croissance la plus rapide. Avec des plugins, comme le « Noise Remover » ou le « Voice leveler » , toujours plus nombreux et rassemblés en une seule interface, le « Audio Clean-up Assistant » , l’ERA Bundle s’impose peu à peu comme un sérieux concurrent à iZotope RX dans le domaine des logiciels de nettoyage et restauration sonore. La simplicité d’utilisation des plugins (souvent munis d’un simple potard à tourner), permet aux créateurs débutants et professionnels d’améliorer instantanément leurs enregistreme

In [4]:
index = minsearch.Index(
    text_fields=['title', 'tags', 'chunk_text'],
    keyword_fields=['article_id', 'category', 'chunk_id']
)

In [5]:
index.fit(documents)

# RAG flow

In [10]:
from openai import OpenAI

client = OpenAI()

In [14]:
query = 'De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?'

In [6]:
response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": query}]
    )
    
response.choices[0].message.content

"Créer une musique dans un home-studio nécessite un certain équipement pour garantir une qualité audio satisfaisante. Voici une liste des matériels essentiels pour commencer :\n\n### 1. **Ordinateur**\n   - Un ordinateur puissant (PC ou Mac) capable de faire tourner un logiciel de production audio (DAW).\n\n### 2. **Logiciel de Production Audio (DAW)**\n   - Choisissez un logiciel tel que Ableton Live, FL Studio, Logic Pro, Cubase, ou Pro Tools.\n\n### 3. **Interface Audio**\n   - Permet de connecter des instruments et des microphones à votre ordinateur tout en assurant une meilleure qualité sonore.\n\n### 4. **Moniteurs de Studio**\n   - Des enceintes de monitoring pour écouter votre musique de manière précise (par exemple, KRK, Yamaha, ou JBL).\n\n### 5. **Casque Audio**\n   - Un bon casque de studio pour un suivi audio précis (par exemple, Sennheiser, Audio-Technica).\n\n### 6. **Microphone**\n   - Un micro de bonne qualité pour enregistrer des voix ou des instruments (par exemple, 

In [11]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=30
    )

    return results

In [12]:
prompt_template = """
You're an audio engineer and sound designer instructor for beginners.
You're particularly specialized in audio home-studio set-up, computer music production and audio post-production in general (editing, mixing and mastering). 
Answer the QUESTION based on the CONTEXT from our arsonor knowledge database (articles).
Use only the facts from the CONTEXT when answering the QUESTION.
Finally add in your response the top 3 articles of arsonor (refer to the 'title') that are the best to read for answering this question.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
article title: {title}
article keywords: {tags}
content: {chunk_text}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [15]:
search_results = search(query)
prompt = build_prompt(query, search_results)

In [16]:
print(prompt)

You're an audio engineer and sound designer instructor for beginners.
You're particularly specialized in audio home-studio set-up, computer music production and audio post-production in general (editing, mixing and mastering). 
Answer the QUESTION based on the CONTEXT from our arsonor knowledge database (articles).
Use only the facts from the CONTEXT when answering the QUESTION.
Finally add in your response the top 3 articles of arsonor (refer to the 'title') that are the best to read for answering this question.

QUESTION: De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?

CONTEXT:
article title: Comment bien débuter en MAO: le home-studio démystifié
article keywords: DAW, hardware, home-studio, MAO, matériel audio, plugin, software
content: trouveras toujours quelqu’un pour te montrer ses 50 synthétiseurs analogiques, ses 10 boîtes à rythmes, ses 3 ordinateurs et quelques interfaces audio multi-pistes avec leurs micros, le tout pour plus de 10000€. La vérité :

In [17]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [19]:
question = 'De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?'
answer = rag(question)
print(answer)

Pour créer votre musique dans un home-studio, voici les équipements essentiels dont vous aurez besoin :

1. **Un ordinateur** : C'est le cœur de votre home-studio. Que vous ayez un Mac ou un PC, assurez-vous que votre appareil soit relativement récent avec un processeur rapide et suffisamment de RAM (4 Go et plus). Il est conseillé de configurer cet ordinateur uniquement pour la musique afin de maximiser les performances.

2. **Une DAW (Digital Audio Workstation)** : C’est le logiciel qui vous permettra de créer, d'éditer et de mixer votre musique. Des options populaires incluent Ableton Live, FL Studio, et GarageBand pour les débutants.

3. **Un casque de qualité** : Un bon casque audio est crucial pour commencer à bien entendre votre musique. Cela vous permettra de vous familiariser avec les sons et de travailler efficacement dans votre DAW.

4. **Vos oreilles !** : C’est l’outil le plus important que vous ayez. Écoutez attentivement et apprenez à identifier les détails dans votre mi

# Retrieval evaluation

In [69]:
df_question = pd.read_csv('../data/ground-truth-data.csv')
df_question.head()

,question,category,chunk,article
0,Quel est l'impact de l'intelligence artificiel...,LA POST-PROD,4615db39-1,4615db39
1,Comment l'IA influence-t-elle la production mu...,LA POST-PROD,4615db39-1,4615db39
2,Quelles sont les tendances des outils audio ut...,LA POST-PROD,4615db39-1,4615db39
3,En quoi l'IA permet-elle aux amateurs d'obteni...,LA POST-PROD,4615db39-1,4615db39
4,Quels avantages l'IA offre-t-elle aux artistes...,LA POST-PROD,4615db39-1,4615db39


In [70]:
ground_truth = df_question.to_dict(orient='records')
ground_truth[0]

{'question': "Quel est l'impact de l'intelligence artificielle dans la post-production audio?",
 'category': 'LA POST-PROD',
 'chunk': '4615db39-1',
 'article': '4615db39'}

In [71]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

### Evaluation based on the chunk_id

In [72]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [73]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['chunk']
        results = search_function(q)
        relevance = [d['chunk_id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [26]:
from tqdm.auto import tqdm

In [74]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/2945 [00:00<?, ?it/s]

{'hit_rate': 0.5297113752122241, 'mrr': 0.3079710027757569}

### Evaluation based on the article_id

In [60]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['article']
        results = search_function(q)
        
        # Ensure that only the first relevant result counts for the MRR
        relevance = []
        seen_article = False
        for d in results:
            if d['article_id'] == doc_id and not seen_article:
                relevance.append(True)
                seen_article = True
            else:
                relevance.append(False)

        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


In [61]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/2945 [00:00<?, ?it/s]

{'hit_rate': 0.5911714770797962, 'mrr': 0.4871826070552721}

### Find the best boost parameters

In [75]:
df_validation = df_question[:100]
df_test = df_question[100:]

In [76]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [77]:
gt_val = df_validation.to_dict(orient='records')

In [78]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [80]:
param_ranges = {
    'title': (0.0, 3.0),
    'tags': (0.0, 3.0),
    'chunk_text': (0.0, 3.0)
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [81]:
simple_optimize(param_ranges, objective, n_iterations=20)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

({'title': 1.482365404824786,
  'tags': 0.3110799058056106,
  'chunk_text': 2.914247315795288},
 0.49071825396825397)

In [82]:
def minsearch_improved(query):
    boost = {
        'title': 1.48,
        'tags': 0.31,
        'chunk_text': 2.91
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/2945 [00:00<?, ?it/s]

{'hit_rate': 0.702546689303905, 'mrr': 0.4231697523378335}

# RAG evaluation

In [83]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [84]:
len(ground_truth)

2945

In [85]:
record = ground_truth[0]
record

{'question': "Quel est l'impact de l'intelligence artificielle dans la post-production audio?",
 'category': 'LA POST-PROD',
 'chunk': '4615db39-1',
 'article': '4615db39'}

In [87]:
question = record['question']
answer_llm = rag(question)

'L\'impact de l\'intelligence artificielle (IA) dans la post-production audio est considérable et se manifeste par une amélioration des outils et processus disponibles pour les artistes, producteurs et ingénieurs du son. La démocratisation de l\'accès à des technologies avancées permet à un plus grand nombre de créateurs de produire des œuvres de qualité professionnelle sans nécessiter une formation approfondie en ingénierie du son.\n\nPremièrement, l\'IA facilite la réalisation des tâches complexes et répétitives, permettant ainsi aux artistes et producteurs de se concentrer sur les aspects créatifs de leur travail. Par exemple, des plugins d\'IA peuvent s\'occuper de tâches telles que le nettoyage et la séparation des pistes audio de manière rapide et efficace, ce qui allège la charge de travail des ingénieurs du son. Des technologies avancées comme le dé-mixage, où une piste audio est décomposée en différentes stems (voix, batterie, etc.), sont déjà courantes et en constante amélior

In [88]:
print(answer_llm)

L'impact de l'intelligence artificielle (IA) dans la post-production audio est considérable et se manifeste par une amélioration des outils et processus disponibles pour les artistes, producteurs et ingénieurs du son. La démocratisation de l'accès à des technologies avancées permet à un plus grand nombre de créateurs de produire des œuvres de qualité professionnelle sans nécessiter une formation approfondie en ingénierie du son.

Premièrement, l'IA facilite la réalisation des tâches complexes et répétitives, permettant ainsi aux artistes et producteurs de se concentrer sur les aspects créatifs de leur travail. Par exemple, des plugins d'IA peuvent s'occuper de tâches telles que le nettoyage et la séparation des pistes audio de manière rapide et efficace, ce qui allège la charge de travail des ingénieurs du son. Des technologies avancées comme le dé-mixage, où une piste audio est décomposée en différentes stems (voix, batterie, etc.), sont déjà courantes et en constante amélioration grâ

In [89]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: Quel est l'impact de l'intelligence artificielle dans la post-production audio?
Generated Answer: L'impact de l'intelligence artificielle (IA) dans la post-production audio est considérable et se manifeste par une amélioration des outils et processus disponibles pour les artistes, producteurs et ingénieurs du son. La démocratisation de l'accès à des technologies avancées permet à un plus grand nombre de créateurs de produire des œuvres de qualité professionnelle sans nécessiter une formation approfondie en ingénierie du son.

Premièrement, l'IA facilite la réalisation des tâches complexes et répétitives, permettant ainsi aux artistes et producteurs de se concentrer sur les aspects créatifs de 

In [90]:
import json

In [91]:
df_sample = df_question.sample(n=200, random_state=1)

In [92]:
sample = df_sample.to_dict(orient='records')

In [93]:
evaluations = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [94]:
evaluations[0]

({'question': "Pourquoi le 32 bits float ne suffit pas avant l'enregistrement dans la DAW ?",
  'category': 'LE HOME STUDIO',
  'chunk': 'e55c4a41-6',
  'article': 'e55c4a41'},
 'Le format 32 bits float est très utile pour le traitement audio au sein d\'une station de travail audionumérique (DAW) car il offre une large plage dynamique et permet d’éviter le clipping pendant les manipulations internes. Cependant, il ne suffit pas avant l\'enregistrement dans la DAW pour plusieurs raisons.\n\nTout d\'abord, l\'avantage du 32 bits float ne s\'applique pas à la phase de l\'enregistrement, qui se déroule encore dans le domaine analogique. Pendant cette étape, il est crucial de s\'assurer que le signal audio ne dépasse pas le 0 dBFS pour éviter la saturation. De plus, une gestion rigoureuse des niveaux est essentielle dès le moment de l\'enregistrement pour maintenir une bonne qualité audio et éviter des problèmes de bruit de quantification lors des conversions numériques.\n\nEnsuite, une foi

In [95]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['chunk'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [96]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.985
PARTLY_RELEVANT    0.015
Name: proportion, dtype: float64

In [97]:
df_eval.to_csv('../data/rag-eval-gpt-4o-mini.csv', index=False)

In [99]:

df_eval[df_eval.relevance == 'PARTLY_RELEVANT']

,answer,id,question,relevance,explanation
18,Pour changer le pitch naturellement dans Ablet...,173567a9-2,Quelle méthode peut-on utiliser pour changer l...,PARTLY_RELEVANT,The generated answer discusses changing pitch ...
31,Pour éviter les pièges du make-up gain lors de...,584d0437-3,Quels conseils professionnels peut on suivre p...,PARTLY_RELEVANT,The generated answer discusses make-up gain an...
146,La question concernant l'activation du mode Th...,173567a9-11,Quelles sont les étapes pour activer le mode T...,PARTLY_RELEVANT,The generated answer acknowledges the question...
